In [1]:
# # 압축파일 풀기
# !unzip data/larger_dataset_with_ICBHI.zip -d data/ICBHI_database

In [2]:
import os
import gc
from time import time

import natsort
from itertools import count
# import torch
# from pydub import AudioSegment, silence

# Math
from scipy.io import wavfile
import librosa
import numpy as np
from scipy import signal
from scipy.signal import find_peaks

# # Visualization
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D

import librosa.display
from IPython.display import Audio
from IPython.display import HTML

import itertools
import noisereduce as nr
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import sklearn

# keras module
import tensorflow as tf
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, Conv2D, MaxPooling1D, Embedding, LSTM, Bidirectional, SimpleRNN, GRU

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# Hyperparameters
freq_lim = 2048

In [3]:
import os

from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import mglearn
from os.path import join

from sklearn.model_selection import train_test_split, train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler, label_binarize, OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer,accuracy_score,f1_score,roc_auc_score,roc_curve,confusion_matrix,auc
from sklearn.metrics import RocCurveDisplay,ConfusionMatrixDisplay
# import umap
from sklearn.manifold import TSNE

In [4]:
tf.__version__

'2.1.0'

In [5]:
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2518496707027860875,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10269412333464437645
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9569401596706835270
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:1"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11100839013144696641
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:2"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7721027403578253651
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:3"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18386846825326975089
 physical_device_desc: "device: XLA_GPU device",

In [7]:
for i in range(2):
    gc.collect()
gc.collect()

0

# Funcition definition 

In [8]:
def make_train_dataset_sklearn(file_path, ordered_file_list, n_mels, frame_sec, step_sec, label_ratio):
    
    data_list = []
    label_list = []
    
    pitch_var = [-3.5, -2, 2, 3.5]
    augmented_number = len(pitch_var)

    for index, file in enumerate(ordered_file_list):
        
        if file[-3:] == 'wav':

            # data load 
            sample_rate = librosa.get_samplerate(file_path + '/' + file)
            # calculated values
            frame_length = int(frame_sec * sample_rate)
            frame_step = int(step_sec * sample_rate)
            data, data_sr = librosa.load(file_path + '/' + file, sr = sample_rate)
            
            for pitch in pitch_var:
                
                ps_data = librosa.effects.pitch_shift(data, sr = sample_rate, n_steps = pitch)

                # mel spectrogram extraction
                melS = librosa.feature.melspectrogram(y = ps_data, sr=sample_rate, n_mels=n_mels,
                                                      hop_length=frame_step, win_length=frame_length)
                melS = librosa.power_to_db(melS, ref=np.max)

                data_list.append(melS.T)
                
            # normal and wheeze split
            normal_event = []
            wheeze_event = []
                
            label = pd.read_csv(file_path + '/' + file[:-3] + 'txt', sep='\t', header=None)
            
            sec_to_index_start = label[0]*sample_rate
            sec_to_index_end = label[1]*sample_rate
            sec_to_index_start = sec_to_index_start.to_numpy().round()
            sec_to_index_end = sec_to_index_end.to_numpy().round()

            for i, val in enumerate(label[2].to_numpy()):
                if val == 'normal':
                    normal_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)),
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

                elif val == 'wheeze':
                    wheeze_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)),
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

            normal_event = np.array(normal_event)
            wheeze_event = np.array(wheeze_event)
            
            y = np.zeros(len(data_list[-1]))
 
            # label ratio decision
            for j in normal_event:
                start = j[0]
                end = j[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 1

            for k in wheeze_event:
                start = k[0]
                end = k[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 2
                
            for number in range(augmented_number):
                label_list.append(y)
                
    X = np.concatenate(data_list, axis = 0)
#     X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.concatenate(label_list, axis = 0)
#     y = utils.to_categorical(y)
    
    return X, y


def make_validation_dataset_sklearn(file_path, ordered_file_list, n_mels, frame_sec, step_sec, label_ratio):

    data_list = []
    label_list = []

    for index, file in enumerate(ordered_file_list):
        if file[-3:] == 'wav':
            
            # data load 
            sample_rate = librosa.get_samplerate(file_path + '/' + file)
            data, data_sr = librosa.load(file_path + '/' + file, sr = sample_rate)

            # framing condition
            frame_length = int(frame_sec * sample_rate)
            frame_step = int(step_sec * sample_rate)

            # mel spectrogram extraction
            melS = librosa.feature.melspectrogram(y = data, sr=sample_rate, n_mels=n_mels,
                                                  hop_length=frame_step, win_length=frame_length)
            melS = librosa.power_to_db(melS, ref=np.max)
#             melS = librosa.power_to_db(melS, ref=np.median)

            data_list.append(melS.T)

    # X must be extracted first

    for index, file in enumerate(ordered_file_list):

        if file[-3:] == 'txt':

            # normal and wheeze split
            normal_event = []
            wheeze_event = []

            label = pd.read_csv(file_path + '/' + file, sep='\t', header=None)

            sec_to_index_start = label[0]*sample_rate
            sec_to_index_end = label[1]*sample_rate
            sec_to_index_start = sec_to_index_start.to_numpy().round()
            sec_to_index_end = sec_to_index_end.to_numpy().round()

            for i, val in enumerate(label[2].to_numpy()):
                if val == 1:
                    normal_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)), 
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

                elif val == 2:
                    wheeze_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)), 
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

            normal_event = np.array(normal_event)
            wheeze_event = np.array(wheeze_event)

            y = np.zeros(len(data_list[int(index/2)]))

            # label ratio decision

            for j in normal_event:
                start = j[0]
                end = j[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 1
#                 y[end:end+delta] = 1


            for k in wheeze_event:
                start = k[0]
                end = k[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 2
#                 y[end:end+delta] = 2


            label_list.append(y)
    
    X = np.concatenate(data_list, axis = 0)
    X = X.reshape(X.shape[0], X.shape[1])
#     X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.concatenate(label_list, axis = 0)
#     y = utils.to_categorical(y)
    
    return X, y


def make_train_dataset_keras(file_path, ordered_file_list, n_mels, frame_sec, step_sec, label_ratio):
    
    data_list = []
    label_list = []
    
    pitch_var = [-3.5, -2, 2, 3.5]
    augmented_number = len(pitch_var)

    for index, file in enumerate(ordered_file_list):
        
        if file[-3:] == 'wav':

            # data load 
            sample_rate = librosa.get_samplerate(file_path + '/' + file)
            # calculated values
            frame_length = int(frame_sec * sample_rate)
            frame_step = int(step_sec * sample_rate)
            data, data_sr = librosa.load(file_path + '/' + file, sr = sample_rate)
            
            for pitch in pitch_var:
                
                ps_data = librosa.effects.pitch_shift(data, sr = sample_rate, n_steps = pitch)

                # mel spectrogram extraction
                melS = librosa.feature.melspectrogram(y = ps_data, sr=sample_rate, n_mels=n_mels,
                                                      hop_length=frame_step, win_length=frame_length)
                melS = librosa.power_to_db(melS, ref=np.max)

                data_list.append(melS.T)
                
            # normal and wheeze split
            normal_event = []
            wheeze_event = []
                
            label = pd.read_csv(file_path + '/' + file[:-3] + 'txt', sep='\t', header=None)
            
            sec_to_index_start = label[0]*sample_rate
            sec_to_index_end = label[1]*sample_rate
            sec_to_index_start = sec_to_index_start.to_numpy().round()
            sec_to_index_end = sec_to_index_end.to_numpy().round()

            for i, val in enumerate(label[2].to_numpy()):
                if val == 'normal':
                    normal_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)),
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

                elif val == 'wheeze':
                    wheeze_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)),
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

            normal_event = np.array(normal_event)
            wheeze_event = np.array(wheeze_event)
            
            y = np.zeros(len(data_list[-1]))
 
            # label ratio decision
            for j in normal_event:
                start = j[0]
                end = j[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 1

            for k in wheeze_event:
                start = k[0]
                end = k[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 2
                
            for number in range(augmented_number):
                label_list.append(y)
                
    X = np.concatenate(data_list, axis = 0)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.concatenate(label_list, axis = 0)
    y = utils.to_categorical(y)
    
    return X, y


def make_validation_dataset_keras(file_path, ordered_file_list, n_mels, frame_sec, step_sec, label_ratio):


    data_list = []
    label_list = []

    for index, file in enumerate(ordered_file_list):
        if file[-3:] == 'wav':

            # data load 
            sample_rate = librosa.get_samplerate(file_path + '/' + file)
            data, data_sr = librosa.load(file_path + '/' + file, sr = sample_rate)

            # framing condition
            frame_length = int(frame_sec * sample_rate)
            frame_step = int(step_sec * sample_rate)

            # mel spectrogram extraction
            melS = librosa.feature.melspectrogram(y = data, sr=sample_rate, n_mels=n_mels,
                                                  hop_length=frame_step, win_length=frame_length)
            melS = librosa.power_to_db(melS, ref=np.max)
#             melS = librosa.power_to_db(melS, ref=np.median)

            data_list.append(melS.T)

    # X must be extracted first

    for index, file in enumerate(ordered_file_list):

        if file[-3:] == 'txt':

            # normal and wheeze split
            normal_event = []
            wheeze_event = []

            label = pd.read_csv(file_path + '/' + file, sep='\t', header=None)

            sec_to_index_start = label[0]*sample_rate
            sec_to_index_end = label[1]*sample_rate
            sec_to_index_start = sec_to_index_start.to_numpy().round()
            sec_to_index_end = sec_to_index_end.to_numpy().round()

            for i, val in enumerate(label[2].to_numpy()):
                if val == 1:
                    normal_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)), 
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

                elif val == 2:
                    wheeze_event.append([int(sec_to_index_start[i]//(step_sec*sample_rate)), 
                                         int(sec_to_index_end[i]//(step_sec*sample_rate))])

            normal_event = np.array(normal_event)
            wheeze_event = np.array(wheeze_event)

            y = np.zeros(len(data_list[int(index/2)]))

            # label ratio decision

            for j in normal_event:
                start = j[0]
                end = j[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 1
#                 y[end:end+delta] = 1


            for k in wheeze_event:
                start = k[0]
                end = k[1]
                delta = int((end-start)*label_ratio)
                y[end-delta:end] = 2
#                 y[end:end+delta] = 2


            label_list.append(y)
    
    X = np.concatenate(data_list, axis = 0)
    X = X.reshape(X.shape[0], X.shape[1])
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.concatenate(label_list, axis = 0)
    y = utils.to_categorical(y)
    
    return X, y


def make_test_dataset_keras(test_signal, test_label, n_mels, frame_sec, step_sec):
    
    # test signal must be noise filtered

    # framing condition
    frame_length = int(frame_sec * sample_rate)
    frame_step = int(step_sec * sample_rate)

    # mel spectrogram extraction
    melS = librosa.feature.melspectrogram(y = test_signal, sr=sample_rate, 
                                          n_mels=n_mels, hop_length=frame_step, win_length=frame_length)
    melS = librosa.power_to_db(melS, ref=np.max)
#     melS = librosa.power_to_db(melS, ref=np.median)

    X_test = melS.T
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)
    
    sec_to_index = test_label[0]*sample_rate
    event_index = sec_to_index//(step_sec*sample_rate)

    y_test = np.zeros(len(X_test))

    for i in event_index:
        y_test[int(i)] = 2
        
    return X_test, y_test


def wheeze_per_resp_acc(y_pred, y_test):
    
    peaks, _ = find_peaks(y_pred)

    after_peaks = peaks[1:]
    before_peaks = peaks[:-1]

    term = after_peaks - before_peaks

    start = 0
    pred_resp_count = 0
    pred_normal_count = 0
    pred_wheeze_count = 0

    for index, val in enumerate(term):
        if val >= 100:
            pred_resp_count += 1
            thres = y_pred[peaks[start:index+1]].mean()
            if thres > 1.0:
                pred_wheeze_count += 1
            elif thres == 1.0 : 
                pred_normal_count += 1
            start = index+1
    
    print('prediction')
    print(pred_wheeze_count, '/', pred_resp_count)
    
    peaks, _ = find_peaks(y_test)

    after_peaks = peaks[1:]
    before_peaks = peaks[:-1]

    term = after_peaks - before_peaks

    start = 0
    test_resp_count = 0
    test_normal_count = 0
    test_wheeze_count = 0

    for index, val in enumerate(term):
        if val >= 100:
            test_resp_count += 1
            thres = y_test[peaks[start:index+1]].mean()
            if thres > 1.0:
                test_wheeze_count += 1
            elif thres == 1.0 : 
                test_normal_count += 1
            start = index+1
             
    print('test')
    print(test_wheeze_count, '/', test_resp_count)
    
    return pred_wheeze_count, pred_resp_count

In [9]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=5, scoring_fit='roc_auc',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
#         cv=cv, 
        n_jobs=1, 
#         scoring=scoring_fit,
#         verbose=2
    )
    
    gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = gs.predict_proba(X_test_data)
    else:
        pred = gs.predict(X_test_data)
    
    return gs, pred

## fixed parameter results

In [10]:
file_path= 'data/train_with_ICBHI'
file_list = os.listdir(file_path)
ordered_file_list = natsort.natsorted(file_list)
ordered_file_list

['103_2b2_Ar_mc_LittC2SE.txt',
 '103_2b2_Ar_mc_LittC2SE.wav',
 '106_2b1_Pr_mc_LittC2SE.txt',
 '106_2b1_Pr_mc_LittC2SE.wav',
 '121_1b1_Tc_sc_Meditron.txt',
 '121_1b1_Tc_sc_Meditron.wav',
 '133_2p4_Al_mc_AKGC417L.txt',
 '133_2p4_Al_mc_AKGC417L.wav',
 '134_2b3_Ar_mc_LittC2SE.txt',
 '134_2b3_Ar_mc_LittC2SE.wav',
 '141_1b2_Lr_mc_LittC2SE.txt',
 '141_1b2_Lr_mc_LittC2SE.wav',
 '143_1b1_Al_sc_Meditron.txt',
 '143_1b1_Al_sc_Meditron.wav',
 '144_1b1_Al_sc_Meditron.txt',
 '144_1b1_Al_sc_Meditron.wav',
 '151_2p2_Pl_mc_AKGC417L.txt',
 '151_2p2_Pl_mc_AKGC417L.wav',
 '152_1b1_Al_sc_Meditron.txt',
 '152_1b1_Al_sc_Meditron.wav',
 '153_1b1_Al_sc_Meditron.txt',
 '153_1b1_Al_sc_Meditron.wav',
 '159_1b1_Pr_sc_Meditron.txt',
 '159_1b1_Pr_sc_Meditron.wav',
 '160_1b2_Ar_mc_AKGC417L.txt',
 '160_1b2_Ar_mc_AKGC417L.wav',
 '171_1b1_Al_sc_Meditron.txt',
 '171_1b1_Al_sc_Meditron.wav',
 '178_1b2_Tc_mc_AKGC417L.txt',
 '178_1b2_Tc_mc_AKGC417L.wav',
 '183_1b1_Tc_sc_Meditron.txt',
 '183_1b1_Tc_sc_Meditron.wav',
 '184_1b

In [11]:
# ordered_file_list = ordered_file_list[1:]
# ordered_file_list

In [12]:
val_file_path= 'data/validation'

val_file_list = os.listdir(val_file_path)
val_ordered_file_list = natsort.natsorted(val_file_list)

val_ordered_file_list

['normal_littmann_1cycle.txt',
 'normal_littmann_1cycle.wav',
 'normal_sim_2cycles.txt',
 'normal_sim_2cycles.wav',
 'wheeze_clinical_validation.txt',
 'wheeze_clinical_validation.wav',
 'wheeze_expiratory_littmann_1cycle.txt',
 'wheeze_expiratory_littmann_1cycle.wav',
 'wheeze_low_pitch_littmann_1cycle.txt',
 'wheeze_low_pitch_littmann_1cycle.wav',
 'wheeze_monophonic_littmann_1cycle.txt',
 'wheeze_monophonic_littmann_1cycle.wav',
 'wheeze_sim_2cycles.txt',
 'wheeze_sim_2cycles.wav']

In [13]:
# val_ordered_file_list = val_ordered_file_list[1:]
# val_ordered_file_list

In [14]:
n_mels = 128
frame_sec = 0.025
step_sec = 0.01
label_ratio = 1

X, y = make_train_dataset_sklearn(file_path, ordered_file_list, n_mels, frame_sec, step_sec, label_ratio)
X_test, y_test = make_validation_dataset_sklearn(val_file_path, val_ordered_file_list, n_mels, frame_sec, step_sec, label_ratio)
#%% Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
# n_timesteps, n_features, n_outputs = X.shape[1], X.shape[2], y.shape[1]

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [16]:
X.shape

(332720, 128)

## Random Forest Classifier

### with k-fold, scaler 

In [17]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [38]:
kfold = KFold(n_splits=10, shuffle=False)

cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))  
#     clf = RandomForestClassifier(random_state=0)
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)

test with fold: 1 : 0.7964955518153403
test with fold: 2 : 0.8473190670834335
test with fold: 3 : 0.7742546285164703
test with fold: 4 : 0.8007934599663381
test with fold: 5 : 0.7973972108679971
test with fold: 6 : 0.789522721808127
test with fold: 7 : 0.8045503726857418
test with fold: 8 : 0.7824897811974032
test with fold: 9 : 0.7970966578504448
test with fold: 10 : 0.4298809810050493


In [39]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

1
0.8362732919254658
acc: 0.8362732919254658
f1: 0.8414140628249657
roc_auc: 0.9555164463021505
[[0.85278569 0.00244002 0.1447743 ]
 [0.08094435 0.82293423 0.09612142]
 [0.06988695 0.1274409  0.80267215]]


In [40]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib')

['saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib']

### with k-fold, without scaler 

In [31]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [32]:
kfold = KFold(n_splits=10, shuffle=False)

cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1

    clf = RandomForestClassifier(random_state=0)
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)

test with fold: 1 : 0.6521699927867276
test with fold: 2 : 0.6149615292137534
test with fold: 3 : 0.6173960086559269
test with fold: 4 : 0.7603991344073094
test with fold: 5 : 0.606275547006492
test with fold: 6 : 0.7246333253185862
test with fold: 7 : 0.718321711949988
test with fold: 8 : 0.7499398893964896
test with fold: 9 : 0.7640358259196922
test with fold: 10 : 0.3539913440730945


In [33]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

8
0.8452173913043478
acc: 0.8452173913043478
f1: 0.8509455354273302
roc_auc: 0.960877767175696
[[0.85563237 0.00569337 0.13867426]
 [0.04721754 0.82799325 0.12478921]
 [0.04727646 0.12332991 0.82939363]]


In [34]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib')

['saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib']

### with k-fold, shuffle, scaler

In [41]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))  
#     clf = RandomForestClassifier(random_state=0)
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)

test with fold: 1 : 0.8239360423178649
test with fold: 2 : 0.8215616734792017
test with fold: 3 : 0.8230944938687185
test with fold: 4 : 0.8235152680932917
test with fold: 5 : 0.8211108439528733
test with fold: 6 : 0.825649194517913
test with fold: 7 : 0.817534263044001
test with fold: 8 : 0.8235753786968021
test with fold: 9 : 0.8233048809810051
test with fold: 10 : 0.8202993508054821


In [42]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

5
0.8370186335403726
acc: 0.8370186335403726
f1: 0.842056913078042
roc_auc: 0.952309017929985
[[0.85278569 0.00244002 0.1447743 ]
 [0.08768971 0.82630691 0.08600337]
 [0.0647482  0.1315519  0.8036999 ]]


In [43]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_shuffle_without_scaler.joblib')

['saved_model/230730_RFC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_shuffle_without_scaler.joblib']

### without k-fold, scaler 

In [56]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [57]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.8372670807453416


In [58]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.8372670807453416
f1: 0.8421785447225325
roc_auc: 0.9530288238899
[[0.85359902 0.00325336 0.14314762]
 [0.08768971 0.82124789 0.09106239]
 [0.06988695 0.12435766 0.8057554 ]]


In [59]:
import joblib
joblib.dump(clf, 'saved_model/230730_RFC_larger_database_ICBHI_ref_max_no_scaler.joblib')

['saved_model/230730_RFC_larger_database_ICBHI_ref_max_no_scaler.joblib']

### without k-fold, with scaler 

In [ ]:
for i in range(2):
    gc.collect()
gc.collect()

In [27]:
clf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.8357763975155279


In [31]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.8357763975155279
f1: 0.8409018850854336
roc_auc: 0.9533355171262423
[[0.84953233 0.00325336 0.14721431]
 [0.08600337 0.82630691 0.08768971]
 [0.0688592  0.12435766 0.80678314]]


In [32]:
import joblib
joblib.dump(clf, 'saved_model/230730_RFC_larger_database_ICBHI_ref_max_with_scaler.joblib')

['saved_model/230730_RFC_larger_database_ICBHI_ref_max_with_scaler.joblib']

## Nearset Neighbors

### with k-fold, scaler

In [18]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [38]:
kfold = KFold(n_splits=10, shuffle=False)
cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = make_pipeline(StandardScaler(), KNeighborsClassifier(3))      
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)
    
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

test with fold: 1 : 0.7484671796104833
test with fold: 2 : 0.7974873767732628
test with fold: 3 : 0.7285104592450108
test with fold: 4 : 0.7365652801154123
test with fold: 5 : 0.7112887713392643
test with fold: 6 : 0.7180512142341909
test with fold: 7 : 0.7118297667708584
test with fold: 8 : 0.729321952392402
test with fold: 9 : 0.7372565520557827
test with fold: 10 : 0.3754207742245732
1
0.7642236024844721


In [39]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

1
0.7642236024844721
acc: 0.7642236024844721
f1: 0.73753981842823
roc_auc: 0.7864435686152825
[[0.97763318 0.00732005 0.01504677]
 [0.3693086  0.53625632 0.09443508]
 [0.42857143 0.20760534 0.36382323]]


In [40]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_KNN_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib')

['saved_model/230730_KNN_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib']

### with k-fold, without scaler

In [35]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [23]:
kfold = KFold(n_splits=10, shuffle=False)
cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = KNeighborsClassifier(3)     
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)
    
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

test with fold: 1 : 0.6234972349122385
test with fold: 2 : 0.5769115171916326
test with fold: 3 : 0.5654003366193796
test with fold: 4 : 0.6816542438086078
test with fold: 5 : 0.4851827362346718
test with fold: 6 : 0.5840947343111325
test with fold: 7 : 0.5925703294061072
test with fold: 8 : 0.6820149074296706
test with fold: 9 : 0.6846597739841308
test with fold: 10 : 0.29868958884347196
8
0.8233540372670808


In [24]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

8
0.8233540372670808
acc: 0.8233540372670808
f1: 0.8269936614885541
roc_auc: 0.9202266846916052
[[0.85237902 0.01220008 0.1354209 ]
 [0.10455312 0.84317032 0.05227656]
 [0.10894142 0.15313464 0.73792395]]


In [25]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_KNN_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib')

['saved_model/230730_KNN_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib']

### without k-fold, with scaler

In [19]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [20]:
clf = make_pipeline(StandardScaler(), KNeighborsClassifier(3))    
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.7607453416149068


In [21]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.7607453416149068
f1: 0.7331757895074731
roc_auc: 0.7802691847024038
[[0.97681985 0.00772672 0.01545344]
 [0.37099494 0.52613828 0.10286678]
 [0.44295992 0.19938335 0.35765673]]


In [22]:
import joblib
joblib.dump(clf, 'saved_model/230730_KNN_larger_database_ICBHI_ref_max_with_scaler.joblib')

['saved_model/230730_KNN_larger_database_ICBHI_ref_max_with_scaler.joblib']

### without k-fold, scaler

In [23]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [24]:
clf = KNeighborsClassifier(3)    
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.8260869565217391


In [25]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.8260869565217391
f1: 0.8294645570249439
roc_auc: 0.9202688934836653
[[0.85725905 0.00732005 0.1354209 ]
 [0.10455312 0.84317032 0.05227656]
 [0.11408016 0.14902364 0.7368962 ]]


In [26]:
import joblib
joblib.dump(clf, 'saved_model/230730_KNN_larger_database_ICBHI_ref_max_without_scaler.joblib')

['saved_model/230730_KNN_larger_database_ICBHI_ref_max_without_scaler.joblib']

## Gaussian Process Classifier

### with k-fold, scaler 

In [17]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [18]:
kfold = KFold(n_splits=10, shuffle=False)
cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = make_pipeline(StandardScaler(), GaussianProcessClassifier(1.0 * RBF(1.0), random_state=0)) 
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)
    
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

MemoryError: Unable to allocate 668. GiB for an array with shape (299448, 299448) and data type float64

In [ ]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

In [ ]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_GPC_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib')

### without k-fold, with scaler 

In [ ]:
for i in range(2):
    gc.collect()
gc.collect()

In [ ]:
clf = make_pipeline(StandardScaler(), GaussianProcessClassifier(random_state=0)) 
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

In [ ]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

In [ ]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_GPC_larger_database_ICBHI_ref_max_with_scaler.joblib')

### without k-fold, scaler

In [ ]:
for i in range(2):
    gc.collect()
gc.collect()

In [ ]:
clf = GaussianProcessClassifier(random_state=0)
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

In [ ]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

In [ ]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_GPC_larger_database_ICBHI_ref_max_without_scaler.joblib')

## Multi Layer Perceptron 

### with k-fold,scaler 

In [22]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [24]:
kfold = KFold(n_splits=10, shuffle=False)
cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = make_pipeline(StandardScaler(), MLPClassifier(early_stopping=True, shuffle=False, random_state=0, verbose=0)) 
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)
    
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

test with fold: 1 : 0.7944517912959846
test with fold: 2 : 0.8429911036306804
test with fold: 3 : 0.77028732868478
test with fold: 4 : 0.7786727578744891
test with fold: 5 : 0.7797246934359221
test with fold: 6 : 0.7704075498918009
test with fold: 7 : 0.7631041115652801
test with fold: 8 : 0.7354832892522241
test with fold: 9 : 0.7980283722048569
test with fold: 10 : 0.387953835056504
1
0.7741614906832298


In [25]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

1
0.7741614906832298
acc: 0.7741614906832298
f1: 0.7561563232858209
roc_auc: 0.8383847354529177
[[0.96990647 0.01098007 0.01911346]
 [0.19561551 0.46711636 0.33726813]
 [0.37101747 0.16238438 0.46659815]]


In [26]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_MLP_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib')

['saved_model/230730_MLP_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_with_scaler.joblib']

### with k-fold without scaler 

In [26]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [27]:
kfold = KFold(n_splits=10, shuffle=False)
cv_model = []
cv_score = []

fold_number = 0

for train_index, val_index in kfold.split(X):
    
    fold_number += 1
    
    clf = MLPClassifier(early_stopping=True, shuffle=False, random_state=0, verbose=0)
    
    train_x, val_x = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]
    
    clf.fit(train_x, train_y)
    score = clf.score(val_x, val_y)
    
    cv_model.append(clf)
    cv_score.append(score)
    
    print('test with fold:', fold_number, ':', score)
    
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

test with fold: 1 : 0.6268634287088243
test with fold: 2 : 0.6179370040875211
test with fold: 3 : 0.588092089444578
test with fold: 4 : 0.7578143784563597
test with fold: 5 : 0.5489300312575138
test with fold: 6 : 0.6513885549410916
test with fold: 7 : 0.6453474392882904
test with fold: 8 : 0.7183517672517432
test with fold: 9 : 0.7030235633565761
test with fold: 10 : 0.3172637653282039
3
0.8683229813664596


In [28]:
best_index = np.argmax(cv_score)
print(best_index)
print(cv_model[best_index].score(X_test, y_test))

y_pred = cv_model[best_index].predict(X_test)
y_prob = cv_model[best_index].predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

3
0.8683229813664596
acc: 0.8683229813664596
f1: 0.8704820265679196
roc_auc: 0.9490209603170922
[[0.91541277 0.01952013 0.0650671 ]
 [0.03541315 0.70320405 0.2613828 ]
 [0.06372045 0.08633094 0.84994861]]


In [29]:
import joblib
joblib.dump(cv_model[best_index], 'saved_model/230730_MLP_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib')

['saved_model/230730_MLP_larger_database_ICBHI_ref_max_cv_fold_10_train_with_9_without_scaler_fix_sample_rate_error.joblib']

### without k-fold, scaler, fix sample rate error 

In [85]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [86]:
clf = MLPClassifier(early_stopping=True, shuffle=False, random_state=0, verbose=0)
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.7803726708074534


In [87]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.7803726708074534
f1: 0.790658200471098
roc_auc: 0.9441566107889893
[[0.78527857 0.00610004 0.20862139]
 [0.07588533 0.59865093 0.32546374]
 [0.05138746 0.06988695 0.87872559]]


### without k-fold, scaler 

In [27]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [28]:
clf = MLPClassifier(early_stopping=True, shuffle=False, random_state=0, verbose=0)
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.7669565217391304


In [29]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.7669565217391304
f1: 0.7684368659991588
roc_auc: 0.9213942807368825
[[0.83326556 0.         0.16673444]
 [0.12310287 0.37605396 0.50084317]
 [0.08633094 0.07605344 0.83761562]]


In [30]:
import joblib
joblib.dump(clf, 'saved_model/230730_MLP_larger_database_ICBHI_ref_max_without_scaler.joblib')

['saved_model/230730_MLP_larger_database_ICBHI_ref_max_without_scaler.joblib']

### without k-fold, with scaler 

In [31]:
for i in range(2):
    gc.collect()
gc.collect()

0

In [32]:
clf = make_pipeline(StandardScaler(), MLPClassifier(early_stopping=True, shuffle=False, random_state=0, verbose=0))
clf.fit(X, y)
score = clf.score(X_test, y_test)
print(score)

0.6581366459627329


In [33]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

acc =accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='weighted')

y_true = utils.to_categorical(y_test)

roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
cm = confusion_matrix(y_test, y_pred, normalize='true')

print('acc:', acc)
print('f1:', f1)
print('roc_auc:', roc_auc)
print(cm)

acc: 0.6581366459627329
f1: 0.6897669924392718
roc_auc: 0.8452456502867675
[[0.71492477 0.27206181 0.01301342]
 [0.02866779 0.72175379 0.24957841]
 [0.15724563 0.36690647 0.47584789]]


In [34]:
import joblib
joblib.dump(clf, 'saved_model/230730_MLP_larger_database_ICBHI_ref_max_with_scaler.joblib')

['saved_model/230730_MLP_larger_database_ICBHI_ref_max_with_scaler.joblib']